In [30]:
import pandas as pd

df = pd.read_csv('data.csv')


In [ ]:
df = df.drop(['End_Lat', 'End_Lng'], axis=1)

df['Start_Time'] = df['Start_Time'].str.split('.').str[0]
df['End_Time']   = df['End_Time'].str.split('.').str[0]

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df = df[df['Start_Time'].dt.year >= 2023]
df = df[df['End_Time'].dt.year >= 2023]

new_df = df.dropna() #Removes all data with an empty cell

new_df.to_csv('cleandata.csv', index=False)

print('num rows after cleaning', len(new_df))

df = pd.read_csv('cleandata.csv')

num rows after cleaning 1871679


In [32]:

df = pd.read_csv('cleandata.csv')



In [33]:
latlng = df[['Start_Lat', 'Start_Lng']]
array = latlng.to_numpy()
print('have all values?', len(array))
#https://stackoverflow.com/questions/31789160/convert-select-columns-in-pandas-dataframe-to-numpy-array

have all values? 1871679


In [34]:
import json
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for lat, lng in array:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [lng, lat]
        },
        "properties": {}
    }
    geojson["features"].append(feature)

with open("data.json", "w") as f:
    json.dump(geojson, f, indent=2)

print('the length of array', len(array), 'should match length of features', len(geojson["features"]))

#https://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file

the length of array 1871679 should match length of features 1871679
